In [277]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from random import shuffle

In [278]:
def efficient_frontier(returns, n_samples=50, gamma_low=-1, gamma_high=10):
    """
    construye un conjunto de problemas de programación cuádrática
    para inferir la frontera eficiente de Markovitz. 
    En cada problema el parámetro gamma se cambia para aumentar
    la penalización del riesgo en la función de maximización.
    """
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  w >= 0]) 
    # Equivalente 
    #prob = cp.Problem(cp.Minimize(risk - gamma*ret), 
    #                  [cp.sum(w) == 1,  w >= 0])   
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []    
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)   
    return ret_data, risk_data, gamma_vals, portfolio_weights


def optimal_portfolio(returns):
    """
    A partir del calculo de la frontera eficiente de Markowitz
    devuelve el portfolio optimo: su ratio de sharpe, rentabilidad,
    riesgo y pesos.
    """
    ret_data, risk_data, _, portfolio_weights = efficient_frontier(returns)

    sharpes = ret_data/risk_data 
    idx = np.argmax(sharpes)
    optimal_ret, optimal_risk = ret_data[idx], risk_data[idx]
    optimal_portfolio = pd.Series(portfolio_weights[idx],
                                index=returns.columns).abs().round(3)

    optimal_portfolio = optimal_portfolio.div(optimal_portfolio.sum())
                                
    return sharpes[idx], optimal_ret, optimal_risk, optimal_portfolio

In [279]:
funds = pd.read_csv('funds.csv', index_col=0, parse_dates=[0])
funds

,AT0000494893,AT0000495189,AT0000495197,AT0000495304,AT0000497268,AT0000497284,AT0000499785,AT0000506050,AT0000506068,AT0000607254,...,US9229086296,US9229086379,US9229086528,US9229087369,US9229087443,US9229087518,US9229087690,US97717W4226,US97717X5941,VGG4157S1048
2020-01-03,240.95,8.55,11.23,170.99,283.58,2.60,13.79,9.31,11.86,135.74,...,141.33,126.08,94.95,166.17,95.32,121.33,136.90,18.0118,33.48,1122.29
2020-01-06,240.95,8.55,11.23,170.99,283.58,2.60,13.79,9.31,11.86,135.74,...,141.33,126.08,94.95,166.17,95.32,121.33,136.90,18.0118,33.48,1120.12
2020-01-07,230.73,8.55,11.23,169.50,280.11,2.60,13.62,9.31,11.86,136.39,...,141.33,126.08,94.95,166.17,95.32,121.33,136.90,18.0118,33.48,1122.76
2020-01-08,234.45,8.55,11.23,169.69,282.22,2.60,13.71,9.31,11.86,136.22,...,141.33,126.08,94.95,166.17,95.32,121.33,136.90,18.0118,33.48,1123.03
2020-01-09,237.00,8.55,11.23,169.60,282.27,2.60,13.72,9.31,11.86,136.02,...,141.33,126.08,94.95,166.17,95.32,121.33,136.90,18.0118,33.48,1127.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-07,185.34,9.08,11.92,149.72,292.03,3.22,12.19,10.35,13.18,140.82,...,204.36,173.28,159.53,247.93,118.41,187.99,191.26,27.9554,33.48,1167.13
2020-12-08,185.34,9.08,11.92,149.72,292.03,3.22,12.19,10.35,13.18,140.82,...,205.28,173.84,161.08,248.55,118.90,189.63,192.04,28.0735,33.48,1167.71
2020-12-09,186.89,9.09,11.94,149.32,293.39,3.19,12.19,10.37,13.20,141.54,...,203.21,172.16,158.56,244.11,118.97,188.51,190.27,28.2213,33.48,1169.33
2020-12-10,188.30,9.10,11.95,150.02,294.73,3.23,12.25,10.37,13.21,141.54,...,203.21,172.16,158.56,244.11,118.97,188.51,190.27,28.2213,33.48,1165.63


In [280]:
returns = np.log(funds).diff().dropna()
returns

,AT0000494893,AT0000495189,AT0000495197,AT0000495304,AT0000497268,AT0000497284,AT0000499785,AT0000506050,AT0000506068,AT0000607254,...,US9229086296,US9229086379,US9229086528,US9229087369,US9229087443,US9229087518,US9229087690,US97717W4226,US97717X5941,VGG4157S1048
2020-01-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.001935
2020-01-07,-0.043341,0.000000,0.000000,-0.008752,-0.012312,0.000000,-0.012404,0.000000,0.000000,0.004777,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.002354
2020-01-08,0.015994,0.000000,0.000000,0.001120,0.007505,0.000000,0.006586,0.000000,0.000000,-0.001247,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000240
2020-01-09,0.010818,0.000000,0.000000,-0.000531,0.000177,0.000000,0.000729,0.000000,0.000000,-0.001469,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.003609
2020-01-10,0.060257,0.000000,0.000000,0.000118,0.017315,0.000000,0.006538,0.000000,0.000000,-0.001913,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-07,-0.003393,0.001102,0.000839,0.006500,0.010499,0.012500,0.004110,0.003872,0.003801,-0.000994,...,-0.001907,-0.000635,0.003642,0.003839,-0.006314,-0.002815,-0.000888,0.008698,0.0,0.000060
2020-12-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004492,0.003227,0.009669,0.002498,0.004130,0.008686,0.004070,0.004216,0.0,0.000497
2020-12-09,0.008328,0.001101,0.001676,-0.002675,0.004646,-0.009360,0.000000,0.001931,0.001516,0.005100,...,-0.010135,-0.009711,-0.015768,-0.018025,0.000589,-0.005924,-0.009260,0.005251,0.0,0.001386
2020-12-10,0.007516,0.001100,0.000837,0.004677,0.004557,0.012461,0.004910,0.000000,0.000757,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.003169


# ALGORTIMO GENETICO

In [283]:
# Generar aleatoriamente una población inicial

# Repetir hasta que se cumpla la condición de parad

#     Evaluar los individuos según la función objetivo (maximizar beneficio, minimizar distancia)

#     Seleccionar padres (en función de la evaluación anterior).
    
#     Cruzamiento: recombinar material genético de los padres

#     Evaluar si el individuo, o parte de su código genético muta

#     Reemplazo generacional
    
#     Generación = Generación +1

# Cuando se cumpla condición de parada, devolver mejor solución.


In [345]:
# genes = activos
n_genes = returns.shape[1]

# cromosomas = inversores
n_cromosomas = round(n_genes * (50/(5+n_genes)))

print(n_genes, n_cromosomas)

68151 50


### Generacion aleatoria inicial

In [346]:
# Generate array of random numbers between 1 and 20
n_activos_init = np.random.randint(1, 21, n_cromosomas)

list_info_cromo = []

# Para cada cromosoma de la población inicial escogo aleatoriamente una cantidad de activos
# entre 1 y 20, y calculo la cartera que maximiza el sharpe. 
for i in n_activos_init:
    returns_random = returns.sample(n=i,axis='columns')
    sharpe, ret, std, weights = optimal_portfolio(returns_random)
    list_info_cromo.append([sharpe, ret, std, weights])


# Lista con los sharpes
sharpes = [item[0] for item in list_info_cromo]
# Normalizo sharpe con el minimo y maximo
norm_sharpes = ((sharpes - min(sharpes))/(max(sharpes) - min(sharpes))).tolist()
# Agrego la lista de norm_sharpes a la lista de cromosomas con el resto de info
list_info_cromo = [[norm_sharpe, list_info[0], list_info[1], list_info[2], list_info[3]] for norm_sharpe, list_info in zip(norm_sharpes, list_info_cromo)]


In [417]:
# Ahora voy generando padres, para posteriormente generar los hijos
for crom in range(int(n_cromosomas/2)):
    # Reordeno la lista de cromosomas aleatoriamente
    shuffle(list_info_cromo)
    # Obtengo una lista con probabilidades de seleccionar a un padre
    rand_probs = np.random.uniform(0, 1, size=len(list_info_cromo))
    # A los padres les asigno un 1 si se seleccionaron, 0 si no
    padres = [1 if rand_prob < list_info[0] else 0 for rand_prob, list_info in zip(rand_probs, list_info_cromo)]

    # Ahora voy a seleccionar los padres que sean 1
    new_padres = []
    [new_padres.append(list_info) if padre == 1 else '' for padre, list_info in zip(padres, list_info_cromo)]
    # Me quedo con los 2 primeros padres
    padres = new_padres[:2]

padres

[[0.6751935952007082,
  0.2404333434589066,
  0.00035021284621207917,
  0.0014565901766114041,
  LU1651443415    0.000
  FR0011269067    0.000
  LU0236489893    0.000
  LU1775415596    0.000
  LU0381555043    0.155
  GB00B1XMSZ01    0.008
  LU1494628131    0.525
  GB0007502080    0.062
  LU0353281818    0.000
  LU1718847640    0.000
  LU0862302675    0.000
  LU1769944528    0.012
  LU0641092787    0.000
  LU1407888996    0.238
  dtype: float64],
 [1.0,
  0.37712727871857327,
  2.6632948160661196e-05,
  7.062058266152557e-05,
  LU0933168436    0.997003
  LU0987096491    0.000000
  HK0000352382    0.000000
  GB00B2990B27    0.001998
  LU0858288946    0.000000
  IE00B179D857    0.000000
  GB00B1PRW734    0.000999
  LU0781589642    0.000000
  LU1762220934    0.000000
  dtype: float64]]